In [ ]:
import math
import uproot
import awkward as ak
import numpy as np

import pyhepmc as hp
from particle import Particle

# Download files

In [ ]:
#!xrdcp root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTZToLLNuNu_M-10_TuneCP5CR1_13TeV-amcatnlo-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v2/2520000/FD6F2D88-5C5C-B844-BA34-B1D2411B0427.root .

In [ ]:
# ------------------------
# Helpers
# ------------------------
# Is 1e6 the right way to go?
############ PRODUCES 0 for the energy of the beam
############ SHOULD WE GET THIS FROM LHE?
def four_vector_from_pt_eta_phi_m(pt, eta, phi, mass, eta_max=10):
    """
    Build (px, py, pz, E) from (pt, eta, phi, mass).
    To avoid overflow in sinh/cosh for crazy |eta|, we clip |eta| to eta_max.
    """
    # NumPy handles both scalars and arrays here
    eta_clipped = np.clip(eta, -eta_max, eta_max)

    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    pz = pt * np.sinh(eta_clipped)

    # E^2 = p^2 + m^2, with pT * cosh(eta)
    e2 = (pt * np.cosh(eta_clipped)) ** 2 + mass ** 2

    if np.isscalar(e2):
        if e2 < 0:
            e2 = 0.0
        e = math.sqrt(e2)
    else:
        e2 = np.where(e2 < 0, 0.0, e2)
        e = np.sqrt(e2)

    return px, py, pz, e


def infer_mass(pdg_id, stored_mass):
    """
    Use stored mass when > 0, otherwise fall back to PDG mass (via particle package).
    You can skip this and just return stored_mass if you prefer the NanoAOD value.
    """
    if stored_mass > 0:
        return stored_mass
    try:
        p = Particle.from_pdgid(int(pdg_id))
        if p.mass is None:
            return 0.0
        # particle gives mass in MeV
        return p.mass / 1000.0
    except Exception:
        return 0.0


# ------------------------
# Converter class
# ------------------------

class NanoAODToHepMCConverter:
    """
    Minimal CMS NanoAOD → HepMC3 converter focused on generator-level info.

    Typical notebook usage:
        conv = NanoAODToHepMCConverter("file.root")
        conv.convert_file("out.hepmc", max_events=100)

    For prototyping individual events:
        arrays = uproot.open("file.root:Events").arrays(conv.required_branches(), library="ak")
        evt0 = conv.build_hepmc_event(arrays, 0)
    """

    def __init__(
        self,
        nanoaod_file: str,
        tree_name: str = "Events",
        use_pdg_masses: bool = True,
    ):
        self.filename = nanoaod_file
        self.tree_name = tree_name
        self.use_pdg_masses = use_pdg_masses

        # Branches we’ll read
        self.genpart_branches = [
            "nGenPart",
            "GenPart_pt",
            "GenPart_eta",
            "GenPart_phi",
            "GenPart_mass",
            "GenPart_pdgId",
            "GenPart_status",
            "GenPart_genPartIdxMother",
        ]

        self.event_branches = [
            "genWeight",
            "Generator_id1",
            "Generator_id2",
            "Generator_scalePDF",
            "Generator_x1",
            "Generator_x2",
            "Generator_xpdf1",
            "Generator_xpdf2",
        ]

        # New: one shared GenRunInfo for all events
        self.run_info = hp.GenRunInfo()
        self.run_info.weight_names = ["genWeight"]
    
    # ------------ public helpers for notebook use ------------

    def required_branches(self):
        """All branch names needed to build a HepMC event."""
        return list(set(self.genpart_branches + self.event_branches))

    def convert_file(
        self,
        output_hepmc_file: str,
        max_events: int | None = None,
        step_size: str | int = "100 MB",
    ):
        """
        Convert all (or first max_events) events from the NanoAOD file to HepMC3.

        Writes an ASCII HepMC3 file using pyhepmc.
        """
        with hp.open(output_hepmc_file, "w") as hepmc_out:
            events_processed = 0

            for arrays in uproot.iterate(
                f"{self.filename}:{self.tree_name}",
                filter_name=self.required_branches(),
                step_size=step_size,
                library="ak",
            ):
                n_events_chunk = len(arrays["nGenPart"])

                for i_evt in range(n_events_chunk):
                    if max_events is not None and events_processed >= max_events:
                        return

                    event = self.build_hepmc_event(arrays, i_evt)
                    hepmc_out.write(event)
                    events_processed += 1

    def build_hepmc_events_from_arrays(self, arrays):
        """
        Convenience: build a list of HepMC events from an awkward record of NanoAOD arrays.
        This is useful if you already have arrays in memory in your notebook.
        """
        n_events = len(arrays["nGenPart"])
        return [self.build_hepmc_event(arrays, i_evt) for i_evt in range(n_events)]

    # ------------ core event builder ------------

    def build_hepmc_event(self, arrays, i_evt: int):
        """
        Build a single HepMC3 GenEvent from an event in the nanoAOD awkward arrays.
        `arrays` is typically one chunk from uproot.iterate or the result of .arrays().
        """
        # Create event (default units: GeV & mm)
        event = hp.GenEvent(
            hp.Units.MomentumUnit.GEV,
            hp.Units.LengthUnit.MM,
        )

        # Attach shared run_info (so writer sees same object each time)
        event.run_info = self.run_info

        
        # === Event weight ===
        if "genWeight" in arrays.fields:
            w = float(arrays["genWeight"][i_evt])
            event.weights = [w]

            # Give the weight a name via run_info
            if event.run_info is None:
                event.run_info = hp.GenRunInfo()
            event.run_info.weight_names = ["genWeight"]

        # === PDF info (if present) ===
        pdf_fields = self.event_branches[1:]  # skip genWeight
        if all(b in arrays.fields for b in pdf_fields):
            pdfinfo = hp.GenPdfInfo()
            pdfinfo.parton_id1 = int(arrays["Generator_id1"][i_evt])
            pdfinfo.parton_id2 = int(arrays["Generator_id2"][i_evt])
            pdfinfo.scale = float(arrays["Generator_scalePDF"][i_evt])
            pdfinfo.x1 = float(arrays["Generator_x1"][i_evt])
            pdfinfo.x2 = float(arrays["Generator_x2"][i_evt])
            pdfinfo.xf1 = float(arrays["Generator_xpdf1"][i_evt])
            pdfinfo.xf2 = float(arrays["Generator_xpdf2"][i_evt])
            event.attributes["GenPdfInfo"] = pdfinfo

        # === GenParticles ===
        pt        = arrays["GenPart_pt"][i_evt]
        eta       = arrays["GenPart_eta"][i_evt]
        phi       = arrays["GenPart_phi"][i_evt]
        mass_arr  = arrays["GenPart_mass"][i_evt]
        pdgId     = arrays["GenPart_pdgId"][i_evt]
        status    = arrays["GenPart_status"][i_evt]
        mother_idx = arrays["GenPart_genPartIdxMother"][i_evt]

        n_genpart = len(pt)
        hepmc_particles = []

        for i in range(n_genpart):
            raw_mass = float(mass_arr[i])
            m_val = (
                infer_mass(int(pdgId[i]), raw_mass)
                if self.use_pdg_masses
                else raw_mass
            )

            px, py, pz, e = four_vector_from_pt_eta_phi_m(
                float(pt[i]),
                float(eta[i]),
                float(phi[i]),
                m_val,
            )

            fv = hp.FourVector(px, py, pz, e)
            p = hp.GenParticle(fv, int(pdgId[i]), int(status[i]))
            event.add_particle(p)
            hepmc_particles.append(p)

        # === Vertices from mother indices ===
        # Map: mother index -> GenVertex
        decay_vertices: dict[int, hp.GenVertex] = {}

        for i in range(n_genpart):
            m = int(mother_idx[i])
            if m < 0 or m >= n_genpart:
                continue  # no valid mother → primordial for now

            if m not in decay_vertices:
                # Place all vertices at origin for now (can use GenVtx later)
                vtx = hp.GenVertex(hp.FourVector(0.0, 0.0, 0.0, 0.0))
                event.add_vertex(vtx)
                # incoming = mother
                vtx.add_particle_in(hepmc_particles[m])
                decay_vertices[m] = vtx
            else:
                vtx = decay_vertices[m]

            # outgoing = this particle
            vtx.add_particle_out(hepmc_particles[i])

        return event


In [ ]:
infilename = 'FD6F2D88-5C5C-B844-BA34-B1D2411B0427.root'

In [ ]:
conv = NanoAODToHepMCConverter(infilename)

In [ ]:
# Convert first 100 events (or drop max_events to do all)
conv.convert_file("test_output.hepmc3", max_events=100)

In [ ]:
# Pull a small batch of events into memory
events_tree = uproot.open(f"{infilename}:Events")
arrays_small = events_tree.arrays(conv.required_branches(), entry_stop=10, library="ak")

# Build HepMC events in memory
hepmc_events = conv.build_hepmc_events_from_arrays(arrays_small)

len(hepmc_events), type(hepmc_events[0])

In [ ]:
hepmc_events[3]

In [ ]:
#hp.listing(hepmc_events[0], precision=2)


# NanoAOD

In [ ]:
gentree_pt = events_tree['GenPart_pt']
gentree_eta = events_tree['GenPart_eta']
gentree_e = events_tree['GenPart_mass']

gentree_status = events_tree['GenPart_status']

In [ ]:
n = 0

print(gentree_pt.array()[0][n])
print(gentree_status.array()[0][n])
print(gentree_eta.array()[0][n])

In [ ]:
max(gentree_eta.array()[:,0])

In [ ]:
eta = 2.4e4
#eta = 100

eta_max = 2.5e4
eta_clipped = np.clip(eta, -eta_max, eta_max)

print(eta_clipped)

np.sinh(eta_clipped)

In [ ]:
events_tree['GenPart_pdgId'].array()[0][0]

In [ ]:
events_tree['GenPart_mass'].array()[0][0]

In [ ]:
events_tree['LHEPart_incomingpz'].array()[0]